In [16]:
import numpy as np
import pandas as pd
import geopandas as gpd

In [17]:
gdf_grid = gpd.read_file('data/Filosofi2017_carreaux_1km_met.gpkg')
gdf_grid.head()

,Idcar_1km,I_est_1km,Ind,Men_1ind,Men_5ind,Men_prop,Men_fmp,Ind_snv,Men_surf,Men_coll,...,Ind_25_39,Ind_40_54,Ind_55_64,Ind_65_79,Ind_80p,Ind_inc,Men_pauv,Men,lcog_geo,geometry
0,CRS3035RES1000mN2029000E4254000,1,2.0,0.3,0.0,0.6,0.0,59349.4,145.1,0.1,...,0.0,0.4,0.3,0.7,0.1,0.0,0.2,0.9,2A041,"POLYGON ((1220028.075 6048968.632, 1219939.34 ..."
1,CRS3035RES1000mN2030000E4251000,1,4.0,0.7,0.1,1.2,0.1,118698.9,290.1,0.2,...,0.1,0.9,0.5,1.4,0.3,0.0,0.3,1.8,2A041,"POLYGON ((1216954.624 6049704.892, 1216865.909..."
2,CRS3035RES1000mN2030000E4252000,1,6.0,0.5,0.2,2.0,0.3,162912.9,334.3,0.3,...,0.5,1.8,1.5,1.2,0.3,0.0,0.3,2.6,2A041,"POLYGON ((1217949.529 6049793.871, 1217860.808..."
3,CRS3035RES1000mN2030000E4253000,1,14.0,1.2,0.4,4.8,0.8,380130.2,780.0,0.8,...,1.2,4.0,3.6,2.8,0.8,0.0,0.8,6.2,2A041,"POLYGON ((1218944.435 6049882.842, 1218855.707..."
4,CRS3035RES1000mN2030000E4254000,1,9.0,1.6,0.2,2.7,0.2,267072.5,652.8,0.3,...,0.2,1.9,1.2,3.3,0.6,0.0,0.8,3.9,2A041,"POLYGON ((1219939.34 6049971.805, 1219850.606 ..."


In [18]:
import folium
from folium import features
import geopandas as gpd
from shapely.geometry import LineString
import branca.colormap as cm

# Filter the data for La Baule-Escoublac
gdf_baule = gdf[gdf['code_insee'] == 44055].copy()

# Convert to GPS coordinates
gdf_baule_gps = gdf_baule.to_crs("EPSG:4326")
ign_geometry_baule_gps = gdf_baule_gps.geometry

# Center the map around La Baule-Escoublac
center_lat, center_lon = gdf_baule_gps.centroid.y.mean(), gdf_baule_gps.centroid.x.mean()
m = folium.Map(location=[center_lat, center_lon], zoom_start=12)

# Add the city contour
folium.GeoJson(
    ign_geometry_baule_gps,
    name="La Baule",
    style_function=lambda x: {"color": "black", "weight": 2, "fillOpacity": 0.1}
).add_to(m)

# Keep only for our area (lcog_geo == '44055')
gdf_grid_baule = gdf_grid[gdf_grid['lcog_geo'] == '44055']

# Find adjacent zones that intersect with the filtered zones
gdf_adjacent_zones = gdf_grid[gdf_grid['lcog_geo'] != '44055']
gdf_adjacent_zones = gpd.sjoin(gdf_adjacent_zones, gdf_grid_baule, how='inner', predicate='intersects')

# Combine the filtered zones and the adjacent zones
gdf_combined = gpd.GeoDataFrame(pd.concat([gdf_grid_baule, gdf_adjacent_zones], ignore_index=True))

# Fill None values in 'Ind' column with a default value (e.g., 0)
gdf_combined['Ind'] = gdf_combined['Ind'].fillna(0)

# Convert to GPS coordinates
gdf_combined_gps = gdf_combined.to_crs("EPSG:4326")

# Create a color map
colormap = cm.linear.YlOrRd_09.scale(gdf_combined_gps['Ind'].min(), gdf_combined_gps['Ind'].max())
colormap.caption = 'Ind'

# Add the grid layer to the map with color based on 'Ind' column
folium.GeoJson(
    gdf_combined_gps,
    name="Grid",
    style_function=lambda feature: {
        'fillColor': colormap(feature['properties']['Ind']),
        'color': 'blue',
        'weight': 1,
        'fillOpacity': 0.6,
    }
).add_to(m)

# Add the color map to the map
colormap.add_to(m)

# Add a layer control
folium.LayerControl().add_to(m)

# Display the map
m.save('map_grid.html')
m

NameError: name 'gdf' is not defined

In [10]:
# Code commune:
df.loc[df["lcog_geo"] == "44055"]

,Idcar_1km,I_est_1km,Ind,Men_1ind,Men_5ind,Men_prop,Men_fmp,Ind_snv,Men_surf,Men_coll,...,Ind_18_24,Ind_25_39,Ind_40_54,Ind_55_64,Ind_65_79,Ind_80p,Ind_inc,Men_pauv,Men,lcog_geo
240695,CRS3035RES1000mN2762000E3389000,0,271.0,75.0,2.0,120.0,3.0,9069299.2,15991.0,100.0,...,8.0,18.9,26.0,47.1,93.0,57.0,0.0,13.0,158.0,44055
240697,CRS3035RES1000mN2762000E3391000,0,1906.5,343.0,41.0,602.0,102.0,49517938.5,86873.4,270.0,...,152.0,289.0,369.0,266.0,340.0,144.0,0.0,97.0,909.0,44055
240698,CRS3035RES1000mN2762000E3392000,0,491.0,75.1,11.0,144.0,20.1,13171739.3,22760.9,96.0,...,36.0,52.9,99.0,118.0,93.0,16.1,0.0,16.0,227.1,44055
241177,CRS3035RES1000mN2763000E3386000,0,197.5,69.0,1.1,89.0,5.0,6335064.8,11589.0,65.9,...,8.1,10.1,24.9,31.9,73.0,33.0,0.0,13.1,122.9,44055
241178,CRS3035RES1000mN2763000E3387000,0,522.5,199.0,1.0,233.0,17.0,15948698.6,27923.8,282.0,...,18.0,22.0,55.0,89.0,181.0,145.0,0.0,37.0,348.0,44055
241179,CRS3035RES1000mN2763000E3388000,0,830.0,253.0,11.0,361.0,21.0,26604113.8,45219.2,291.0,...,42.0,59.0,102.0,113.0,314.0,140.0,1.0,42.0,491.0,44055
241180,CRS3035RES1000mN2763000E3389000,0,1656.0,535.0,12.0,641.0,72.0,48830318.1,85138.9,727.0,...,73.5,130.0,216.0,249.0,537.0,323.0,3.0,89.0,1016.0,44055
241181,CRS3035RES1000mN2763000E3390000,0,553.0,91.9,11.0,190.0,21.0,15381930.5,29385.6,23.0,...,49.9,42.0,108.0,95.0,140.0,58.1,0.0,16.0,262.0,44055
241182,CRS3035RES1000mN2763000E3391000,0,395.0,38.0,8.0,122.9,23.0,10420699.3,18469.2,20.0,...,31.4,63.0,72.0,59.0,83.1,6.1,0.0,5.0,170.0,44055
241183,CRS3035RES1000mN2763000E3392000,1,16.5,6.1,0.0,5.4,0.0,376012.5,1079.0,0.0,...,2.1,2.4,3.4,2.7,2.7,1.5,0.0,1.5,10.0,44055


In [13]:
# To save:
df_clean = df.loc[df["lcog_geo"] == "44055"][["Idcar_1km", "Ind", "Ind_snv"]]
df_clean.to_csv('data/inputs_1km.csv', index=False)

In [14]:
print(len(df_clean))

17
